In [2]:
import pandas as pd
from xml.dom.minidom import Document
import os
import cv2
from pathlib import Path
import glob
from tqdm import tqdm

In [3]:
dict = {'0': "0",  # 字典对类型进行转换
        '1': "1",
        '2': "2",
        '3': "3",
        '4': "4",
        '5': '5',
        '6': "6",
        '7': "7",
        '8': "8",}
dict = {
    '0': 'shoe',  # 鞋子
    '1': 'bag',  # 包包
    '2': 'upperbody',  # 上装
    '3': 'paints',  # 裤子
    '4': 'skirt',  # 裙子
    '5': 'wholebody',  # 连体装
}

### Dataset

In [56]:
!tree -d -L 1 /home/chenyi/workspace/yolov5/datasets/

/home/chenyi/workspace/yolov5/datasets/
├── coco128
├── dadet100kv2
├── dadet100kv3_june
├── dadet100kv4_1_june
├── dadet100kv4_june
├── dadet_gallv4_june
├── dadetv5
├── dadetv5_query
├── dadetv6_1
├── dadetv6_2
├── dadetv6_3
├── dataExps
├── deepedia
├── deepedia_catg6
├── deepedia_catg8
├── haowei_bag_shoe
├── hwpedia
├── hwpedia70k
├── hwpedia80k
├── tuneGallery10k
├── tuneGallery5k
├── tuneQuery10k
└── validation_all

23 directories


In [79]:
folder_name = 'hwpedia'
save_root = Path(f'/home/chenyi/workspace/yolox/datasets/{folder_name}')

In [62]:
!tree -d -L 2 $root

/home/chenyi/workspace/yolov5/datasets/hwpedia
├── images
│   ├── train
│   └── val
└── labels
    ├── train
    └── val

6 directories


In [78]:
names = ['JPEGImages', 'Annotations', 'ImageSets/Main', 'labels']
mk_root = save_root /names[3]
!mkdir -p $mk_root

In [77]:
!tree -d -L 2 $mk_root.parent

/home/chenyi/workspace/yolox/datasets/hwpedia/ImageSets
└── Main

1 directory


In [85]:
dataset_name = 'hwpedia'
picPath = str(save_root /names[0]) + '/'
xmlPath = str(save_root /names[1]) + '/'
train_val_path = str(save_root /names[2]) + '/'
txtPath = str(save_root /names[3]) + '/'

In [88]:
%%time
makexml(txtPath, xmlPath, picPath, dataset=dataset_name)

20it [00:00, 97.56it/s]

20150720135713768.txt


9990it [02:45, 59.56it/s] 

20150720133904463.txt


29262it [09:46, 36.85it/s] 

20150720134705403.txt


29459it [09:50, 54.63it/s]libpng warning: iCCP: known incorrect sRGB profile
65232it [21:25, 42.87it/s] libpng warning: iCCP: known incorrect sRGB profile
70152it [22:58, 68.34it/s] libpng warning: iCCP: known incorrect sRGB profile
74180it [24:17, 50.88it/s] 

CPU times: user 11min 16s, sys: 5min 54s, total: 17min 11s
Wall time: 24min 18s


#### 小数据集生成

In [41]:
root_save = Path('/nas/chenyi/datasets_nas/haowei34k_all/haowei34k_hundred')
label_save = root_save /'labels'
img_save = root_save /'JPEGImages'
xml_save = root_save /'Annotations'

dataset_name = 'haowei34k_hundred'
txtPath = str(label_save) + '/'
picPath = str(img_save) + '/'
xmlPath = str(xml_save) + '/'

In [29]:
root_save = Path('/nas/chenyi/datasets_nas/haowei34k_all/haowei34k_hundred')
label_save = root_save /'labels'
img_save = root_save /'JPEGImages'

In [40]:
import shutil
pathes = glob.glob(txtPath+'*.txt')
pathes = pathes[:100]
# files = [Path(x).name for x in pathes]
for i, txtsp in tqdm(enumerate(pathes)):
    txtsp = Path(txtsp)
    txttp = label_save /txtsp.name
    img_sp = Path(f'{picPath}/{txtsp.stem}.jpg')
    img_tp = img_save /img_sp.name
    if not txttp.parent.is_dir():
        txttp.parent.mkdir()
    if not img_tp.parent.is_dir():
        img_tp.parent.mkdir()
    shutil.copyfile(txtsp, txttp)
    shutil.copyfile(img_sp, img_tp)
    # break

 ... (more hidden) ...


#### TRAIN VAL SET SPLIT.

In [ ]:
import random 

In [92]:
random.seed(0)
xmlfilepath=xmlPath
saveBasePath=train_val_path

In [93]:
trainval_percent=0.8
train_percent=1

temp_xml = os.listdir(xmlfilepath)
total_xml = []
for xml in temp_xml:
    if xml.endswith(".xml"):
        total_xml.append(xml)

In [94]:
num=len(total_xml)  
list=range(num)  
tv=int(num*trainval_percent)  
tr=int(tv*train_percent)  
trainval= random.sample(list,tv)  
train=random.sample(trainval,tr)  

print("train and val size",tv)
print("traub suze",tr)
ftrainval = open(os.path.join(saveBasePath,'trainval.txt'), 'w')  
ftest = open(os.path.join(saveBasePath,'test.txt'), 'w')  
ftrain = open(os.path.join(saveBasePath,'train.txt'), 'w')  
fval = open(os.path.join(saveBasePath,'val.txt'), 'w')  

train and val size 63149
traub suze 63149


In [97]:
for i  in tqdm(list):  
    name=total_xml[i][:-4]+'\n'  
    if i in trainval:  
        ftrainval.write(name)  
        if i in train:  
            ftrain.write(name)  
        else:  
            fval.write(name)  
    else:  
        ftest.write(name)  

ftrainval.close()  
ftrain.close()  
fval.close()  
ftest .close()

100%|█████████████████████████████████████████████████████████████████| 78937/78937 [01:16<00:00, 1038.23it/s]


#### MAKE XML

In [ ]:
# makexml(txtPath, xmlPath, picPath, dataset_name)

In [ ]:
pathes = glob.glob(txtPath+'*.txt')
# pathes = pathes[:10]
files = [Path(x).name for x in pathes]
for i, name in tqdm(enumerate(files)):
    xmlBuilder = Document()
    annotation = xmlBuilder.createElement("annotation")  # 创建annotation标签
    xmlBuilder.appendChild(annotation)
    txtFile = open(txtPath+name)
    txtList = txtFile.readlines()
    try:
        img = cv2.imread(picPath+name[0:-4]+".jpg")
        Pheight, Pwidth, Pdepth=img.shape
    except:
        img = cv2.imread(picPath+name[0:-4]+".jpeg")
        Pheight, Pwidth, Pdepth=img.shape

    flag = 0
    for i in txtList:
        flag += 1
        oneline = i.strip().split(" ")
        folder = xmlBuilder.createElement("folder")  # folder标签
        folderContent = xmlBuilder.createTextNode(dataset)
        folder.appendChild(folderContent)
        annotation.appendChild(folder)
        if flag == 1:
            filename = xmlBuilder.createElement("filename")  # filename标签
            filenameContent = xmlBuilder.createTextNode(name[0:-4]+".jpg")
            filename.appendChild(filenameContent)
            annotation.appendChild(filename)

            size = xmlBuilder.createElement("size")  # size标签
            width = xmlBuilder.createElement("width")  # size子标签width
            widthContent = xmlBuilder.createTextNode(str(Pwidth))
            width.appendChild(widthContent)
            size.appendChild(width)
            height = xmlBuilder.createElement("height")  # size子标签height
            heightContent = xmlBuilder.createTextNode(str(Pheight))
            height.appendChild(heightContent)
            size.appendChild(height)
            depth = xmlBuilder.createElement("depth")  # size子标签depth
            depthContent = xmlBuilder.createTextNode(str(Pdepth))
            depth.appendChild(depthContent)
            size.appendChild(depth)
            annotation.appendChild(size)

        object = xmlBuilder.createElement("object")
        picname = xmlBuilder.createElement("name")
        nameContent = xmlBuilder.createTextNode(dict[oneline[0]])
        picname.appendChild(nameContent)
        object.appendChild(picname)
        pose = xmlBuilder.createElement("pose")
        poseContent = xmlBuilder.createTextNode("Unspecified")
        pose.appendChild(poseContent)
        object.appendChild(pose)
        truncated = xmlBuilder.createElement("truncated")
        truncatedContent = xmlBuilder.createTextNode("0")
        truncated.appendChild(truncatedContent)
        object.appendChild(truncated)
        difficult = xmlBuilder.createElement("difficult")
        difficultContent = xmlBuilder.createTextNode("0")
        difficult.appendChild(difficultContent)
        object.appendChild(difficult)
        bndbox = xmlBuilder.createElement("bndbox")
        xmin = xmlBuilder.createElement("xmin")
        mathData=int(((float(oneline[1]))*Pwidth+1)-(float(oneline[3]))*0.5*Pwidth)
        xminContent = xmlBuilder.createTextNode(str(mathData))
        xmin.appendChild(xminContent)
        bndbox.appendChild(xmin)
        ymin = xmlBuilder.createElement("ymin")
        mathData = int(((float(oneline[2]))*Pheight+1)-(float(oneline[4]))*0.5*Pheight)
        yminContent = xmlBuilder.createTextNode(str(mathData))
        ymin.appendChild(yminContent)
        bndbox.appendChild(ymin)
        xmax = xmlBuilder.createElement("xmax")
        mathData = int(((float(oneline[1]))*Pwidth+1)+(float(oneline[3]))*0.5*Pwidth)
        xmaxContent = xmlBuilder.createTextNode(str(mathData))
        xmax.appendChild(xmaxContent)
        bndbox.appendChild(xmax)
        ymax = xmlBuilder.createElement("ymax")
        mathData = int(((float(oneline[2]))*Pheight+1)+(float(oneline[4]))*0.5*Pheight)
        ymaxContent = xmlBuilder.createTextNode(str(mathData))
        ymax.appendChild(ymaxContent)
        bndbox.appendChild(ymax)
        object.appendChild(bndbox)

        annotation.appendChild(object)

    f = open(xmlPath+name[0:-4]+".xml", 'w')
    xmlBuilder.writexml(f, indent='\t', newl='\n', addindent='\t', encoding='utf-8')
    f.close()

In [87]:
def makexml(txtPath, xmlPath, picPath, dataset='VOC2007'):  # 读取txt路径，xml保存路径，数据集图片所在路径
#     files = os.listdir(txtPath)
    pathes = glob.glob(txtPath+'*.txt')
    pathes = pathes[4760:]
    files = [Path(x).name for x in pathes]
    for i, name in tqdm(enumerate(files)):
        xmlBuilder = Document()
        annotation = xmlBuilder.createElement("annotation")  # 创建annotation标签
        xmlBuilder.appendChild(annotation)
        txtFile = open(txtPath+name)
        txtList = txtFile.readlines()
        try:
            img = cv2.imread(picPath+name[0:-4]+".jpg")
            Pheight, Pwidth, Pdepth=img.shape
        except:
            print(name)
            continue

        flag = 0
        for i in txtList:
            flag += 1
            oneline = i.strip().split(" ")
            folder = xmlBuilder.createElement("folder")  # folder标签
            folderContent = xmlBuilder.createTextNode(dataset)
            folder.appendChild(folderContent)
            annotation.appendChild(folder)
            if flag == 1:
                filename = xmlBuilder.createElement("filename")  # filename标签
                filenameContent = xmlBuilder.createTextNode(name[0:-4]+".jpg")
                filename.appendChild(filenameContent)
                annotation.appendChild(filename)

                size = xmlBuilder.createElement("size")  # size标签
                width = xmlBuilder.createElement("width")  # size子标签width
                widthContent = xmlBuilder.createTextNode(str(Pwidth))
                width.appendChild(widthContent)
                size.appendChild(width)
                height = xmlBuilder.createElement("height")  # size子标签height
                heightContent = xmlBuilder.createTextNode(str(Pheight))
                height.appendChild(heightContent)
                size.appendChild(height)
                depth = xmlBuilder.createElement("depth")  # size子标签depth
                depthContent = xmlBuilder.createTextNode(str(Pdepth))
                depth.appendChild(depthContent)
                size.appendChild(depth)
                annotation.appendChild(size)

            object = xmlBuilder.createElement("object")
            picname = xmlBuilder.createElement("name")
            nameContent = xmlBuilder.createTextNode(dict[oneline[0]])
            picname.appendChild(nameContent)
            object.appendChild(picname)
            pose = xmlBuilder.createElement("pose")
            poseContent = xmlBuilder.createTextNode("Unspecified")
            pose.appendChild(poseContent)
            object.appendChild(pose)
            truncated = xmlBuilder.createElement("truncated")
            truncatedContent = xmlBuilder.createTextNode("0")
            truncated.appendChild(truncatedContent)
            object.appendChild(truncated)
            difficult = xmlBuilder.createElement("difficult")
            difficultContent = xmlBuilder.createTextNode("0")
            difficult.appendChild(difficultContent)
            object.appendChild(difficult)
            bndbox = xmlBuilder.createElement("bndbox")
            xmin = xmlBuilder.createElement("xmin")
            mathData=int(((float(oneline[1]))*Pwidth+1)-(float(oneline[3]))*0.5*Pwidth)
            xminContent = xmlBuilder.createTextNode(str(mathData))
            xmin.appendChild(xminContent)
            bndbox.appendChild(xmin)
            ymin = xmlBuilder.createElement("ymin")
            mathData = int(((float(oneline[2]))*Pheight+1)-(float(oneline[4]))*0.5*Pheight)
            yminContent = xmlBuilder.createTextNode(str(mathData))
            ymin.appendChild(yminContent)
            bndbox.appendChild(ymin)
            xmax = xmlBuilder.createElement("xmax")
            mathData = int(((float(oneline[1]))*Pwidth+1)+(float(oneline[3]))*0.5*Pwidth)
            xmaxContent = xmlBuilder.createTextNode(str(mathData))
            xmax.appendChild(xmaxContent)
            bndbox.appendChild(xmax)
            ymax = xmlBuilder.createElement("ymax")
            mathData = int(((float(oneline[2]))*Pheight+1)+(float(oneline[4]))*0.5*Pheight)
            ymaxContent = xmlBuilder.createTextNode(str(mathData))
            ymax.appendChild(ymaxContent)
            bndbox.appendChild(ymax)
            object.appendChild(bndbox)

            annotation.appendChild(object)

        f = open(xmlPath+name[0:-4]+".xml", 'w')
        xmlBuilder.writexml(f, indent='\t', newl='\n', addindent='\t', encoding='utf-8')
        f.close()